In [7]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from dateutil.parser import parse
import os
from datetime import datetime
import json

In [52]:
# Map Quest Geocoding API 
# https://developer.mapquest.com/documentation/geocoding-api/batch/post/
MAP_QUEST_KEY = "y45T24TbDOgGLOfpIN7AE4Y5MsVZ24Gc"
MAP_QUEST_URL = "http://open.mapquestapi.com/geocoding/v1/address"

def get_lat_long(location): 
    
    url = MAP_QUEST_URL
    querystring = {"key": MAP_QUEST_KEY}
    payload = {
        'location' : location,
        "options": {
            "thumbMaps": False,
            "maxResults": 1
        }
    }
    headers = {
        'Content-Type': "application/json",
        'Accept': "application/json"
    }   
    response = requests.request("POST", url, data=json.dumps(payload), headers=headers, params=querystring)
    data = json.loads(response.text)
    # Now you can access Json 

    #print(json.dumps(data, indent=1))
    return data['results'][0]['locations'][0]['displayLatLng']

In [123]:
res = requests.get('https://covid.gov.pk')
soup = BeautifulSoup(res.text, 'html.parser')
    
locations = soup.findAll("div", {"class": "boxx"})

df = pd.DataFrame()
lst_dict = []

for m_loc in locations:
    loc=m_loc.find("h6", "title").text
    status=m_loc.find("div", "h4").text
    l=get_lat_long(loc)
    nstatus = status.split("\n", 2)
    lst_dict.append({'city':loc, 'lat':l.get('lat'), 'lng': l.get('lng'), 'status': nstatus})
   
 
                    #df.append({'c1':loc,str(l.get('lat')), 'c2':loc,str(l.get('lng')), 'c3': nstatus},ignore_index=True)
df4 = df.append(lst_dict)
print(df4)

          city        lat         lng  \
0        SINDH  25.500000   69.000000   
1          KPK  53.119420   23.125874   
2       PUNJAB  30.929321   75.500484   
3    ISLAMABAD  33.666667   73.166667   
4  BALOCHISTAN  28.000000   66.000000   
5       AJK/GB  39.783730 -100.445882   

                                              status  
0                                          [234,654]  
1                                           [63,339]  
2                                          [149,222]  
3                                           [40,019]  
4                                           [18,569]  
5  [8,606,                                       ...  


In [130]:
import folium as folium
    
# Center Map on State College, PA
center_point = [25.500000,69.000000]
county_map = folium.Map(location=center_point, zoom_start=4)
popup_text = "<div style=\"width:150px;\">{}: {}</div>"

# Display Data Points
for row in df4.itertuples():
    county_map.add_child(
     folium.Marker(
        location=[row.lat, row.lng],
        popup=popup_text.format(row.city, row.status),
        icon=folium.Icon(color="red", icon="info-sign"),
        )
    )
                                 
                                
                                                 
                                                 
    
# Add Category Legend
legend_html = """
<div style="position:fixed;
     bottom: 50px; 
     left: 50px; 
     width: 120px; 
     height: 90px; 
     border:2px solid grey; 
     z-index: 9999;
     font-size:14px;">
     &nbsp;<b>Positive Cases:</b><br>
     &nbsp;<i class="fa fa-circle fa-1x" style="color:green"></i>&nbsp;Under 10<br>
     &nbsp;<i class="fa fa-circle fa-1x" style="color:orange"></i>&nbsp;Under 100<br>
     &nbsp;<i class="fa fa-circle fa-1x" style="color:red"></i>&nbsp;Over 100
</div>"""
    
# Add Legend
county_map.get_root().html.add_child(folium.Element(legend_html))

# Display Map
display(county_map)

In [64]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


'C:\Users\OrhanEyhab\.conda\envs\TensorFlow' is not recognized as an internal or external command,
operable program or batch file.
